# Set Covering - 2023-10-10
Copyright(c) 2023 Alex Buffa


In [208]:
import numpy as np
from random import random
from typing import Tuple, Set
from functools import reduce
from operator import or_
from queue import PriorityQueue, LifoQueue, SimpleQueue, Queue
from collections import namedtuple
from typing import Callable
from math import ceil
from tqdm.notebook import tqdm
Result = namedtuple("Result", ["name", "iters", "taken", "coverage", "prio"])
State = Tuple[Set[int], Set[int]]

Define our problem data

In [209]:
PROBLEM_SIZE = 10
NUM_SETS = 30
THRESHOLD = 0.3
SETS = tuple(np.array([random() < THRESHOLD for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
# Redefine SETS until the problem is solvable
while not all(reduce(or_, [SETS[i] for i in range(NUM_SETS)])):
    SETS = tuple(np.array([random() < THRESHOLD for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
results: dict[str, Result] = dict()


In [210]:
# Utility function just to see our current taken array
def visualize_state(state: State) -> list[int]:
    return sum([SETS[i] for i in state[0]])

In [211]:
def goal_check(state: State):
    return all(reduce(or_, [SETS[i] for i in state[0]], np.array([False for _ in range(PROBLEM_SIZE)])))

In [212]:
def search(name: str, initial_state: State = None,*, frontier: "Queue" = None, priority: Callable[[State],int] = None) -> Result:
    """Generic Search Function.
    Through the parameters 
    """
    if initial_state is None:
        initial_state = (set(), set(range(NUM_SETS)))
    assert len(initial_state) == 2, "Invalid State"
    if frontier is None:
        frontier = PriorityQueue()
    if priority is None:
        priority = lambda _: None
    WrappedState = namedtuple("WrappedState", ["priority", "state"])
    frontier.put(WrappedState(priority(initial_state), initial_state))
    _, state = frontier.get()
    counter = 0
    with tqdm(total=None) as pbar:
        while not goal_check(state):
            counter += 1
            for a in state[1]:
                new_state = (state[0] ^ {a}, state[1] ^ {a})
                frontier.put(WrappedState(priority(new_state), new_state))
            _, state = frontier.get()
            pbar.update()
    res = Result(name, counter, state[0], visualize_state(state), priority(state))
    results[name] = res
    return res


## Depth First Search

In [213]:
search(name="Depth-First", frontier=LifoQueue()).taken

0it [00:00, ?it/s]

{23, 24, 25, 26, 27, 28, 29}

## Breadth First Search

In [214]:
# Using SimpleQueue, which does it internally
search(name="Breadth-First", frontier=SimpleQueue()).taken

0it [00:00, ?it/s]

{1, 2, 21}

## Djikstra Search

In [215]:
def cost(state: State) -> int:
    """Number of sets"""
    return len(state[0]) 

In [216]:
search(name="Djikstra", priority=cost).taken

0it [00:00, ?it/s]

{1, 9, 16}

## A* Search

A* requires a heuristic function that is admissible, i.e. it never overestimates the cost to reach the goal.
For example, we define the distance function as the optimal number of sets that are needed to cover the missing tiles.    
With the above distance function we have an admissible heuristic function.  
The priority for A* is given by the sum of the cost function and the heuristic function.

In [217]:
def distance(state: State) -> int:
    max_size = max(sum(s) for i, s in enumerate(SETS) if i in state[1])
    if(len(state[0]) == 0 ):
        return ceil(PROBLEM_SIZE/max_size)
    return ceil((sum([SETS[i] for i in state[0]]) == 0).sum() / max_size)

In [218]:
search(name="A*", priority=lambda x: cost(x) + distance(x)).taken

0it [00:00, ?it/s]

{1, 2, 21}

In [219]:
def informed_cost(state: State) -> int:
    """Number tiles missing + number of overlapped tiles"""
    return sum(abs(np.ones(PROBLEM_SIZE) - sum([SETS[i] for i in state[0]])))

In [220]:
search("Greedy", priority=informed_cost).taken

0it [00:00, ?it/s]

{1, 8, 16}

In [221]:
print("All the results obtained above, sorted by number of iterations")
for result in reversed(results.values()):
    print(result)

All the results obtained above, sorted by number of iterations
Result(name='Greedy', iters=4, taken={8, 16, 1}, coverage=array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1]), prio=1.0)
Result(name='A*', iters=25, taken={1, 2, 21}, coverage=array([1, 1, 1, 2, 1, 1, 1, 1, 1, 2]), prio=3)
Result(name='Djikstra', iters=1111, taken={16, 9, 1}, coverage=array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2]), prio=3)
Result(name='Breadth-First', iters=1760, taken={1, 2, 21}, coverage=array([1, 1, 1, 2, 1, 1, 1, 1, 1, 2]), prio=None)
Result(name='Depth-First', iters=7, taken={23, 24, 25, 26, 27, 28, 29}, coverage=array([4, 3, 5, 1, 2, 1, 3, 2, 1, 3]), prio=None)
